<a href="https://colab.research.google.com/github/wdconinc/practical-computing-for-scientists/blob/master/Projects/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project #3 - Agent-Based Models: Schelling Model of Segregation

In this project you will explore [agent-based models](https://en.wikipedia.org/wiki/Agent-based_model) in a simulation of dynamics in society. An agent-based model is a class of computational models for simulating the actions and interactions of autonomous agents (both individual or collective entities such as organizations or groups) with a view to assessing their effects on the system as a whole.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

## The Schelling Model

The Schelling (1971) segregation model is a classic of agent-based modeling, demonstrating how agents following simple rules lead to the emergence of qualitatively different macro-level outcomes. Agents are randomly placed on a grid. There are two types of agents, one constituting the majority and the other the minority. All agents want a certain number (generally 3) of their 8 surrounding neighbors to be of the same type in order for them to be happy. Unhappy agents will move to a random available grid space. While individual agents do not have a preference for a segregated outcome (e.g. they would be happy with 3 similar neighbors and 5 different ones), the aggregate outcome is nevertheless heavily segregated.

Reference: [T. Schelling, "Dynamic Models of Segregation," Journal of Mathematical Sociology, 1, 1971 pp. 143–186](https://www.stat.berkeley.edu/~aldous/157/Papers/Schelling_Seg_Models.pdf).

I also want to call out the following two references, by Joanna Schug, associate professor of psychology at W&M:
- [To affinity and beyond! How our preference to be among similar people interacts with our social ecology, The Inquisitive Mind, vol 6, issue 26 (2015)](http://www.in-mind.org/article/to-affinity-and-beyond-how-our-preference-to-be-among-similar-people-interacts-with-our)
- [Schelling's Model of ResidentialSegregation, Wolfram Demonstration](http://demonstrations.wolfram.com/SchellingsModelOfResidentialSegregation/)



In this project we will store the agent-based models on a grid and encode their identity as an integer number (with zero representing an empty location). This will be easier than using a Go board with pennies, which is how Schelling reportedly first did this simulation.

### Initialize the grid of agents

Create a function `initialize_grid(shape, prob)` that fills a grid with shape in argument `shape` (a tuple with two elements) and with a probability in argument `prob` (a list in which all elements sum up to 1). You may find the function `np.random.choice` useful.

#### Test case

With the function you defined above, the following commands should create a grid of $30 \times 30$, filled for 25% with members of group 1 and for 25% with members of group 2.

In [0]:
M = 30
N = 30
P = [0.5, 0.25, 0.25]

# Initialize the grid
grid = initialize_grid((M,N), P)

# Plot the grid as an image
plt.imshow(grid)

### Classify the neighbors

Create a function `neighbors(grid, i, j)` that returns the list of neighbors of each group for location $(i,j)$. The function must return, for example, $[1, 4, 3]$ if there is 1 neighboring location that is empty, 4 neighboring locations that have a member of group 1, and 3 neighboring locations that have a member of group 2.

Note: Assume that the world wraps around at the edges and your world is topologically equivalent to a torus. You can then use `(i + 1) % M` to wrap around past the higher edge. To visualize this, consider this [video of Conway's game of life on a torus](https://www.youtube.com/watch?v=lxIeaotWIks). John Conway's [game of life](https://en.wikipedia.org/wiki/Conway's_Game_of_Life) is one of the the first agent-based models developed.

#### Test case

With the function you defined above, the following commands should return the number 8 (since the values in the list that `neighbors` returns must add up to 8).

In [0]:
print(np.sum(neighbors(grid, 30, 30)))

### Determine happiness of an agent

Create a function `unhappy_lt3(grid, i, j)` that returns `True` if the agent at location $(i,j)$ is unhappy, i.e. when it is surrounded by less than 3 agents of its own group. An empty location will always be happy.

#### Test case

With the function you defined above, the following command should either return `True` or `False`, depending on the group of the agent and its neighbors.

In [0]:
print(grid[1, 1], neighbors(grid, 1, 1), unhappy_lt3(grid, 1, 1))

### Setup some rules for moving unhappy agents

Create a function `move_random(grid, i, j)` that moves the agent at location $(i,j)$ to a random empty position. Your function must return a tuple with the new location. Make sure you explicitly exclude staying in the same place.

Create a function `move_closest(grid, i, j)` that moves the agent at location $(i,j)$ to the closest empty position (and in a random direction if there are multiple). You can use the [Manhattan distance](https://en.wiktionary.org/wiki/Manhattan_distance) to determine the closest empty location. Your function must return a tuple with the new location.

#### Test case

With the functions you defined above, the following command should show the move of an agent from one location to a random location. Note that sometimes this test command will pick an empty cell so nothing will appear to move.

In [0]:
grid_before = grid.copy()
shape = np.shape(grid)
i, j = np.random.randint(shape[0]), np.random.randint(shape[1])
move_random(grid, i, j)
grid_after = grid.copy()
plt.imshow(grid_after - grid_before)

With the functions you defined above, the following command should show the move of an agent from one location to a location nearby. Note that sometimes the test command will pick an empty cell so nothing will appear to move.

In [0]:
grid_before = grid.copy()
shape = np.shape(grid)
i, j = np.random.randint(shape[0]), np.random.randint(shape[1])
move_closest(grid, i, j)
grid_after = grid.copy()
plt.imshow(grid_after - grid_before)

### Run a single step in the agent-based model simulation

Now we develop the rest of the model. Write a function `step(grid, unhappy, move)` that evolves the grid based on the functions `unhappy` and `move` which take the syntax as defined above. The function must return the number of moves.

#### Test case

With the function you defined above, the following command should show the move of a large number of agents from one location to a location nearby. If you keep repeating this command (with Ctrl-Enter) you should see the number of unhappy agents who move steadily decrease.

In [0]:
grid_before = grid.copy()
moves = step(grid, unhappy_lt3, move_closest)
print(moves)
grid_after = grid.copy()
plt.imshow(grid_after - grid_before)
plt.show()
plt.imshow(grid_after)
plt.show()

### Create a happiness metric for all agents

In order to study the dynamics of our population, we need to calculate some metrics. We will start with a simple one: the fraction of happy agents. Create a function `metric_happiness(grid, unhappy)` that returns the fraction of happy agents out of the total of all agents (don't include empty locations).

#### Test case

Test the happiness metric above by calculating how happy your population is.

In [0]:
print("Happiness metric =", 100 * metric_happiness(grid, unhappy_lt3), "%")

### Create the full simulation

Finally, create function `simulate(shape, prob, unhappy, move, metric = None, N = 100)` that initalizes a new grid and evolves it until stable (no moves in a step), or until $N$ steps have been completed. The fuction should return the grid and number of steps.

If the metric is specified, assume is a function with calling syntax `metric(grid, unhappy)` which should be run after each step. A numpy array with the outputs should be returned (suitably shaped in case the metric returns a list or numpy array itself, which you can assume to be 1-dimensional).

#### Test case

Now run the full simulation on a $100 \times 100$ grid starting from a population density of 80%, evenly distributed between two groups, using the default happiness rule (lt3).

In [0]:
grid, steps = simulate((100,100), [0.10, 0.45, 0.45], unhappy_lt3, move_random, N = 200)
print("Number of steps =", steps)
plt.imshow(grid, cmap = 'hot_r')

What do you observe? Describe in a text box. You may find the [Parable of the Polygons](https://ncase.me/polygons/) an apt explanation.

...

### Run simulations with different unhappiness criteria

How is our outcome affected by the happiness rules?
- Write a function `unhappy_lt2` under which an agent will be unhappy when less than 2 of its neighbors is of the same group.
- Write a function `unhappy_lt4` under which an agent will be unhappy when less than 4 of its neighbors is of the same group.

For both cases run a simulation.

We can also create differences in the happiness criteria between groups, though the groups will be less likely to reach an equilibrium due to competing desires. 

## Developing segregation metrics

We introduced the happiness metric above but we didn't look yet at the evolution of it.

In [0]:
grid, steps, happiness_lt3 = simulate((100,100), [0.10, 0.45, 0.45], unhappy_lt3, move_random, metric = metric_happiness, N = 200)
print("Number of steps =", steps)
plt.imshow(grid, cmap = 'hot_r')
plt.show()
plt.plot(happiness_lt3)
plt.xlabel("Iteration")
plt.ylabel("Happiness")
plt.show()

In order to study how the model changes over time, develop at least one other metric. You could consider:
- `metric_homophily(grid,unhappy)`: this returns the number of agents that have only neighbors within their group.
- `metric_islands(grid,unhappy)`: this returns the number of disconnected islands of a single group (agents that are connected at a corner are assumed to form a connection).
- `metric_compactness(grid,unhappy)`: this returns a sorted (large to small) list of the compactness of each population island. You can use any measure of compactness (Convex Hull, Reock, Polsby-Popper, moment of inertia).

## Impact of initial distribution

Now that we have a working simulation, we can study all kinds of things! How does the ultimate happiness depend on the initial distribution of empty, group 1, and group 2 agents? Create a 2-dimensional contour plot in fraction of group 1 and fraction of group 2 agents that shows the asymptotic happiness for the population.

## Impact of different preferences

## Impact of preferring diversity

What happens when agents are unhappy both when surrounded by less than 3 agents of their group AND when there are more than 6 agents of their group (they prefer some diversity)?

## Impact of < your choice >

Perform at least one additional study of a quantity that you find interesting. This could involve:
- Reaching an asymptotic equilibrium in one condition and determining the time to reach equilibrium when the criteria for happiness change.
- Introducing more than two groups in various proportions.
- Introducing some groups that are diversity-averse and some groups that are diversity-seeking.
- Expanding the population from a flat grid to a 3-dimensional grid.

## Using External Software for Agent-Based Modeling (Bonus Points)

Instead of writing the code above, we could have used existing software framworks to perform our simulations above. In particular we could have used [mesa](https://github.com/projectmesa/mesa),  a Python package for agent-based modeling.

Using the mesa framework has some other advantages:
- all agents can be directed to move simultaneously, in random order, or in a predetermined order (which is probably how you implemented your own algorithm above),
- data collectors (what we called metrics above) can be defined within the framework.

### Installing mesa in Google Colaboratory

Let's first install the mesa framework to the Google Colaboratory cloud. We use `pip` for this, which will download and install mesa.

In [0]:
!pip install mesa

As an example of how to use mesa we use the Boltzmann Wealth Model (see for example this [article in the New Scientist](https://www.newscientist.com/article/dn22105-inequality-the-physics-of-our-finances/). This is a simple model of agents exchanging wealth. All agents start with the same amount of money. Every step, each agent with one unit of money or more gives one unit of wealth to another random agent.

We setup an example model `MoneyModel` and agent `MoneyAgent`. This requires that we write two core classes: one for the overall model, the other for the agents. Since we did not write python classes throughout this course, a fully worked out example is provided.

In [0]:
import random

from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum( xi * (N-i) for i,xi in enumerate(x) ) / (N*sum(x))
    return (1 + (1/N) - 2*B)
  
class MoneyModel(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            
        self.datacollector = DataCollector(
            model_reporters = {"Gini": compute_gini},  # A function to call
            agent_reporters = {"Wealth": "wealth"})  # An agent attribute

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

class MoneyAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()

In [0]:
model = MoneyModel(50, 10, 10)

for i in range(100):
    model.step()

We can plot the [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient) which is an indicator of wealth inequality (larger Gini, more inequality). It is defined as the ratio of area A over area A + B in the figure below. Perfect wealth equality corresponds to a Gini coefficient of zero. Perfect wealth inequality (all wealth with 1 agent) corresponds to a Gini coefficient of one.

<img src="https://upload.wikimedia.org/wikipedia/commons/5/59/Economics_Gini_coefficient2.svg" width="50%">

In [0]:
gini = model.datacollector.get_model_vars_dataframe()
gini.plot()

### Implementation of Schelling's model in mesa

Take a look at the model above and adapt it to the Schelling model of segretation. Create at least 1 figure that can be compared to a figure you created above.

## References


https://arxiv.org/pdf/1406.5215.pdf

http://nifty.stanford.edu/2014/mccown-schelling-model-segregation/


